# <font color="#000099">llm-d-benchmarking Sweep Analysis</font>

This notebook imports data from configuration sweeps with [llm-d-benchmark](https://github.com/llm-d/llm-d-benchmark), and creates Pareto plots to compare configurations for a particular model and workload.

The first cell contains function and class definitions to support basic functionality, while the second cell imports data from user-provided directories into [Pandas](https://pandas.pydata.org/) [DataFrames](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

The cells following will look at the different scenarios (a particular selection of model, GPU, and workload input/output size) and create tables and Pareto plots for different configurations under these scenarios.

While this basic functionality may be sufficient for many purposes, this notebook should be considered a starting point for more detailed analysis and customization by the user.

## Package imports and definitions (run once)

In [ ]:
################################################################################
# Package imports
################################################################################

import os
import re
import sys
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import pandas

#sys.path.insert(0, '../workload/report/')
import convert
import schema


class Text:
    """ANSI SGR control codes for text formatting"""
    DEFAULT = "\x1b[0m"
    BOLD = "\x1b[1m"
    BOLD_OFF = "\x1b[22m"
    UNDERLINE = "\x1b[4m"
    UNDERLINE_OFF = "\x1b[24m"
    DEFAULT_COLOR = "\x1b[39m"
    DEFAULT_BG_COLOR = "\x1b[49m"
    RED = "\x1b[31m"
    YELLOW = "\x1b[33m"
    GREEN = "\x1b[32m"
    CYAN = "\x1b[36m"
    BLUE = "\x1b[34m"
    MAGENTA = "\x1b[35m"
    BLACK = "\x1b[30m"
    WHITE = "\x1b[37m"
    BG_RED = "\x1b[41m"
    BG_YELLOW = "\x1b[43m"
    BG_GREEN = "\x1b[42m"
    BG_CYAN = "\x1b[46m"
    BG_BLUE = "\x1b[44m"
    BG_MAGENTA = "\x1b[45m"
    BG_BLACK = "\x1b[40m"
    BG_WHITE = "\x1b[47m"


def warn(mesg: str) -> None:
    """Print a warning message.

    Args:
        mesg (str): Warming message.
    """
    sys.stderr.write(f'{Text.YELLOW}{mesg}\n{Text.DEFAULT}')


def error(mesg: str, err_code: int = 1) -> None:
    """Print an error message and exit with an error code.

    Args:
        mesg (str): Error message.
        err_code (int): Error code.
    """
    sys.stderr.write(f'{Text.RED}{mesg}\n{Text.DEFAULT}')
    sys.exit(err_code)


def check_dir(dir: str) -> None:
    """Print an error if directory does not exist.

    Args:
        dir (str): Directory to check existence of.
    """
    if not os.path.isdir(dir):
        error(f'Invalid path: {dir}')


def check_file(file: str) -> None:
    """Print an error if file does not exist.

    Args:
        file (str): File to check existence of.
    """
    if not os.path.isfile(file):
        error(f'Invalid file: {file}')


def get_results_files(source_dir: str) -> list[str]:
    """Get a list of results files within provided path (recursive).

    Args:
        source_dir (str): Directory to recursively search for results files.
    
    Returns:
        list: List of paths to results files.
    """
    result_files = []
    check_dir(source_dir)
    path = Path(source_dir)
    for file in path.rglob("results_*.yaml"):
        # Make sure filename format matches results_YYYY-MM-DD_hh.mm.ss.yaml
        if not re.search('.*results_[0-9]{4}-[0-9]{2}-[0-9]{2}_[0-9]{2}\\.[0-9]{2}\\.[0-9]{2}\\.yaml', str(file)):
            continue
        result_files.append(str(file))
    return result_files


def get_results_dirs(results_files: list[str]) -> list[str]:
    """Given a list of results files, get the set of parent directories.

    When importing the actual results file, only the newest results file in a
    run directory will be imported.

    Args:
        results_files (list[str]): List of results files.

    Returns:
        list[str]: List of unique parent directories.
    """
    dirs = set([])
    for file in results_files:
        dir = os.path.abspath(file).rsplit(os.sep, 1)[0]
        dirs.add(dir)
    results_dirs = list(dirs)
    results_dirs.sort()
    return results_dirs


def make_benchmark_runs_df() -> pandas.core.frame.DataFrame:
    """Create DataFrame for benchmark run results.

    Returns:
        DataFrame: Empty DataFrame for benchmark runs.
    """
    return pandas.DataFrame(columns=[
        'Name',
        'Directory',
        'Model',
        'GPU',
        'DP',
        'TP',
        'PP',
        'EP',
        'Replicas',
        'P_DP',
        'P_TP',
        'P_PP',
        'P_EP',
        'P_Replicas',
        'D_DP',
        'D_TP',
        'D_PP',
        'D_EP',
        'D_Replicas',
        'Concurrency',
        'ISL',
        'OSL',
        'Backend',
        'Duration',
        'Completed',
        'Request_Throughput',
        'Output_Token_Throughput',
        'Total_Token_Throughput',
        'Mean_TTFT_ms',
        'Mean_TPOT_ms',
        'Mean_ITL_ms',
        'Mean_E2EL_ms',
    ])


def _get_replicas_and_parallelism(result: schema.BenchmarkReport) -> dict[str, int | None]:
    """Get the number of replicas and parallelisms.

    Args:
        result (BenchmarkReport): Benchmark run to evaluate.

    Returns:
        dict[str, int | None]: Replicas and parallelisms for standalone or
            prefill/decode configuration. Irrelevant fields will have a value
            of None.
    """
    rp = {}
    rp['replicas'] = result.scenario.host.type.count(schema.HostType.REPLICA)
    rp['p_replicas'] = result.scenario.host.type.count(schema.HostType.PREFILL)
    rp['d_replicas'] = result.scenario.host.type.count(schema.HostType.DECODE)
    if rp['replicas'] == 0:
        rp['replicas'] = None
    if rp['p_replicas'] == 0:
        rp['p_replicas'] = None
    if rp['d_replicas'] == 0:
        rp['d_replicas'] = None
    rp['tp'] = None
    rp['dp'] = None
    rp['pp'] = None
    rp['ep'] = None
    rp['p_tp'] = None
    rp['p_dp'] = None
    rp['p_pp'] = None
    rp['p_ep'] = None
    rp['d_tp'] = None
    rp['d_dp'] = None
    rp['d_pp'] = None
    rp['d_ep'] = None
    if rp['replicas']:
        # We have a standalone setup
        rp['tp'] = result.scenario.host.accelerator[0].parallelism.tp
        rp['dp'] = result.scenario.host.accelerator[0].parallelism.dp
        rp['pp'] = result.scenario.host.accelerator[0].parallelism.pp
        rp['ep'] = result.scenario.host.accelerator[0].parallelism.ep
        return rp
    # We have a P/D setup
    for ii, accel in enumerate(result.scenario.host.accelerator):
        if result.scenario.host.type[ii] is schema.HostType.PREFILL and not rp['p_tp']:
            rp['p_tp'] = accel.parallelism.tp
            rp['p_dp'] = accel.parallelism.dp
            rp['p_pp'] = accel.parallelism.pp
            rp['p_ep'] = accel.parallelism.ep
        if result.scenario.host.type[ii] is schema.HostType.DECODE and not rp['d_tp']:
            rp['d_tp'] = accel.parallelism.tp
            rp['d_dp'] = accel.parallelism.dp
            rp['d_pp'] = accel.parallelism.pp
            rp['d_ep'] = accel.parallelism.ep
        if rp['p_tp'] and rp['d_tp']:
            break
    return rp


def _make_name(result: schema.BenchmarkReport) -> str:
    """Create a name based on the benchmark run's configuration.

    Args:
        result (BenchmarkReport): Benchmark run to create a name for.

    Returns:
        str: Name of benchmark run, providing replica and parallelism details.
    """
    rp = _get_replicas_and_parallelism(result)
    if rp['replicas']:
        # We have a standalone setup
        return f'{rp['replicas']}R TP{rp['tp']}'
    # We have a P/D setup
    # TODO we currently assume the only type of parallelism is TP
    return f'{rp['p_replicas']}P TP{rp['p_tp']}, {rp['d_replicas']}D TP{rp['d_tp']}'


def add_results_entry_to_df(
    runs_df: pandas.core.frame.DataFrame,
    results_dir: str) -> None:
    """Load a results file and add it to the DataFrame of benchmark runs.

    Args:
        runs_df (DataFrame): DataFrame to add a row to for the provided run.
        results_dir (str): Directory path to get the latest results file from,
            and enter into the provided DataFrame.
    """
    result = convert.import_vllm_benchmark(results_dir)
    rp = _get_replicas_and_parallelism(result)
    runs_df.loc[len(runs_df)] = {
        'Name': _make_name(result),
        # We want the base directory for the sweep, which is two levels up
        'Directory': os.path.abspath(results_dir).rsplit(os.sep, 2)[0],
        'Model': result.scenario.model.name,
        # Assume heterogeneous across P and D
        'GPU': result.scenario.host.accelerator[0].model,
        'DP': rp['dp'],
        'TP': rp['tp'],
        'PP': rp['pp'],
        'EP': rp['ep'],
        'Replicas': rp['replicas'],
        'P_DP': rp['p_dp'],
        'P_TP': rp['p_tp'],
        'P_PP': rp['p_pp'],
        'P_EP': rp['p_ep'],
        'P_Replicas': rp['p_replicas'],
        'D_DP': rp['d_dp'],
        'D_TP': rp['d_tp'],
        'D_PP': rp['d_pp'],
        'D_EP': rp['d_ep'],
        'D_Replicas': rp['d_replicas'],
        # TODO This is specific to vLLM benchmark, will need universal support
        'Concurrency': result.scenario.load.args['max_concurrency'],
        # We need to group by ISL/OSL exactly, so round and convert to int.
        # Round ISL to nearest 10's
        'ISL': int(round(result.metrics.requests.input_length.mean, -1)),
        'OSL': int(round(result.metrics.requests.output_length.mean)),
        'Duration': result.metrics.time.duration,
        'Completed': result.metrics.requests.total,
        'Request_Throughput': result.metrics.throughput.requests_per_sec,
        'Output_Token_Throughput': result.metrics.throughput.output_tokens_per_sec,
        'Total_Token_Throughput': result.metrics.throughput.total_tokens_per_sec,
        'Mean_TTFT_ms': result.metrics.latency.time_to_first_token,
        'Mean_TPOT_ms': result.metrics.latency,
        'Mean_ITL_ms': result.metrics.latency,
        'Mean_E2EL_ms': result.metrics.latency,
    }
    # Add calculated columns
    if rp['tp']:
        runs_df['Is_PD'] = False
        runs_df['Num_GPUs'] = runs_df['TP']*runs_df['Replicas']
    else:
        runs_df['Is_PD'] = True
        runs_df['Num_GPUs'] = runs_df['P_TP']*runs_df['P_Replicas'] + runs_df['D_TP']*runs_df['D_Replicas']
    runs_df['Thpt_per_GPU'] = runs_df['Output_Token_Throughput']/runs_df['Num_GPUs']
    runs_df['Thpt_per_User'] = runs_df['Output_Token_Throughput']/runs_df['Concurrency']


def get_scenarios(runs_df: pandas.core.frame.DataFrame) -> list[tuple[str]]:
    """Get a list of available scenarios from runs DataFrame, where
    configurations and concurrency will be swept.

    Args:
        runs_df (DataFrame): Benchmark runs to find the scenarios for.

    Returns:
        list[tuple[str]]: List of scenarios, consisting of unique groups of
            model, GPU type, ISL, and OSL.
    """
    columns = ['Model', 'GPU', 'ISL', 'OSL']
    return list(set(runs_df.set_index(columns).index))


def print_scenarios(scenarios: list[tuple[str]]) -> None:
    """Print a formatted table of scenarios.

    Args:
        scenarios (list[tuple[str]]): Scenario groups to print.
    """
    columns = ['Model', 'GPU', 'ISL', 'OSL']
    # Get maximum text length for each column, including header
    spans = list(map(len, columns))
    for sc in scenarios:
        for jj, item in enumerate(sc):
            if spans[jj] < len(str(item)):
                spans[jj] = len(str(item))

    # Create header, starting with scenario index
    header = f'{Text.BOLD}{Text.BLUE}IDX  {Text.DEFAULT}{Text.BOLD}'
    # Add each column name to header
    for ii, col in enumerate(columns):
        header += col + " " * (spans[ii] - len(col) + 2)
    header += f'{Text.DEFAULT}'
    print(header)

    # Print details of each scenario
    for ii, sc in enumerate(scenarios):
        row = f'{Text.BLUE}{ii}{Text.DEFAULT}' + " " * (5 - len(str(ii)))
        for jj, val in enumerate(sc):
            row += f'{str(val)}' + " " * (spans[jj] - len(str(val)) + 2)
        print(row)

## Import datasets

In [ ]:
################################################################################
# User inputs
################################################################################

# List of directories containing benchmark sweeps to import.
search_dirs = [
    "/files/",
]

################################################################################
# Standard code
################################################################################

# Create blank DataFrames for benchmarking runs
runs = make_benchmark_runs_df()

# Populate the runs DataFrame
for dir in search_dirs:
    # Find all results files in the directory
    results_files = get_results_files(dir)
    # Get the set of parent directories for the results files. For each parent
    # directory, only the newest results file will be imported.
    results_dirs = get_results_dirs(results_files)
    for results_dir in results_dirs:
        # Import the results and add to the runs DataFrame
        add_results_entry_to_df(runs, results_dir)

## Scenarios available

In [ ]:
# Scenarios available, where model, GPU type, ISL and OSL are constant.
# Configurations (seplicas and parallelism) are swept within a scenario.

scenarios = get_scenarios(runs)
print_scenarios(scenarios)

## Pareto plotting and tables

In [ ]:
################################################################################
# User inputs
################################################################################

# Select scenario
idx = 0

# Show P/D disaggregated scenarios
show_pd = True
# Show standalone scenarios
show_sa = True

# Segregate traces by directory (directories with identical scenarios, such as
# repeated runs, will not be joined together in a single trace)
seg_by_dir = True

################################################################################
# Standard code
################################################################################

# Get parameters of selected scenario
model, gpu, isl, osl = scenarios[idx]

# Filter on column values
pd_runs_selected = runs[
    (runs['Model'] == model) &
    (runs['GPU'] == gpu) &
    (runs['ISL'] == isl) &
    (runs['OSL'] == osl)][[
    'Model',
    'GPU',
    'P_TP',
    'P_Replicas',
    'D_TP',
    'D_Replicas',
    'Concurrency',
    'ISL',
    'OSL',
    'Output_Token_Throughput',
    'Thpt_per_GPU',
    'Thpt_per_User',
    'Directory']].drop('Model', axis=1).drop('GPU', axis=1).drop('ISL', axis=1).drop('OSL', axis=1)#.sort_values(by='Output_Token_Throughput')

sa_runs_selected = runs[
    (runs['Model'] == model) &
    (runs['GPU'] == gpu) &
    (runs['ISL'] == isl) &
    (runs['OSL'] == osl) &
    (runs['Is_PD']) == False][[
    'Model',
    'GPU',
    'TP',
    'Replicas',
    'Concurrency',
    'ISL',
    'OSL',
    'Output_Token_Throughput',
    'Thpt_per_GPU',
    'Thpt_per_User',
    'Directory']].drop('Model', axis=1).drop('GPU', axis=1).drop('ISL', axis=1).drop('OSL', axis=1)#.sort_values(by='Output_Token_Throughput')

# Plot performance results
colors = ['#FF0000', '#FFAA00', '#DDDD00', '#00DD00', '#00FFFF', '#0000FF',
          '#FF00FF', '#666666', '#000000']

# Unique configurations of replicas and TP, described as a tuple
# Tuple format is (rep, tp, p_rep, p_tp, d_rep, d_tp, dir, is_pd)
config_sets = []
if seg_by_dir:
    configs_pd = list(set(pd_runs_selected.set_index(['P_Replicas', 'P_TP', 'D_Replicas', 'D_TP', 'Directory']).index))
    configs_sa = list(set(sa_runs_selected.set_index(['Replicas', 'TP', 'Directory']).index))
    for conf in configs_pd:
        config_sets.append((
            None,    # Replicas
            None,    # TP
            conf[0], # P replicas
            conf[1], # P TP
            conf[2], # D replicas
            conf[3], # D TP
            conf[4], # Directory
            True     # Is PD
        ))
    for conf in configs_sa:
        config_sets.append((
            conf[0], # Replicas
            conf[0], # TP
            None,    # P replicas
            None,    # P TP
            None,    # D replicas
            None,    # D TP
            conf[2], # Directory
            False    # Is PD
        ))
else:
    pd_runs_selected = pd_runs_selected.drop('Directory', axis=1)
    sa_runs_selected = sa_runs_selected.drop('Directory', axis=1)
    configs_pd = list(set(pd_runs_selected.set_index(['P_Replicas', 'P_TP', 'D_Replicas', 'D_TP']).index))
    configs_sa = list(set(sa_runs_selected.set_index(['Replicas', 'TP']).index))
    for conf in configs_pd:
        config_sets.append((
            None,    # Replicas
            None,    # TP
            conf[0], # P replicas
            conf[1], # P TP
            conf[2], # D replicas
            conf[3], # D TP
            None,    # Directory
            True     # Is PD
        ))
    for conf in configs_sa:
        config_sets.append((
            conf[0], # Replicas
            conf[0], # TP
            None,    # P replicas
            None,    # P TP
            None,    # D replicas
            None,    # D TP
            None,    # Directory
            False    # Is PD
        ))

# Sort so prinouts/plots are organized
config_sets.sort()

# Convert the list of sets to a list of dicts, to make code following clearer
configs = []
for conf in config_sets:
    configs.append({
        'rep': conf[0],
        'tp': conf[1],
        'p_rep': conf[2],
        'p_tp': conf[3],
        'd_rep': conf[4],
        'd_tp': conf[5],
        'dir': conf[6],
        'is_pd': conf[7],
    })

# Sweep through configurations
for ii, conf in enumerate(configs):
    is_pd = 'P_TP' in conf
    # Make a DataFrame for specific configuration
    if conf['is_pd']:
        # This configuration is PD
        if seg_by_dir:
            conf_df = pd_runs_selected[
                (pd_runs_selected['P_Replicas'] == conf['p_rep']) &
                (pd_runs_selected['P_TP'] == conf['p_tp']) &
                (pd_runs_selected['D_Replicas'] == conf['d_rep']) &
                (pd_runs_selected['D_TP'] == conf['d_tp']) &
                (pd_runs_selected['Directory'] == conf['dir'])
            ].sort_values(by='Concurrency')
        else:
            conf_df = pd_runs_selected[
                (pd_runs_selected['P_Replicas'] == conf['p_rep']) &
                (pd_runs_selected['P_TP'] == conf['p_tp']) &
                (pd_runs_selected['D_Replicas'] == conf['d_rep']) &
                (pd_runs_selected['D_TP'] == conf['d_tp'])
            ].sort_values(by='Concurrency')

        # Print table
        display(conf_df)
    
        # Plot throughputs for configuration
        plt.plot(conf_df.Thpt_per_User, conf_df.Thpt_per_GPU,
                 label=f'{conf['p_rep']}P-TP{conf['p_tp']} {conf['d_rep']}D-TP{conf['d_tp']}',
                 marker='o', markersize=4,
                 color=colors[ii%len(colors)]
                )
        for jj, val in enumerate(conf_df.Concurrency):
            plt.text(list(conf_df.Thpt_per_User)[jj],
                     list(conf_df.Thpt_per_GPU)[jj]+pd_runs_selected['Thpt_per_GPU'].max()*0.02,
                     str(val), ha='center', color=colors[ii%len(colors)])
    else:
        # This configuration is standalone
        if seg_by_dir:
            conf_df = sa_runs_selected[
                (sa_runs_selected['Replicas'] == conf['rep']) &
                (sa_runs_selected['TP'] == conf['tp']) &
                (sa_runs_selected['Directory'] == conf['dir'])
            ].sort_values(by='Concurrency')
        else:
            conf_df = sa_runs_selected[
                (sa_runs_selected['Replicas'] == conf['rep']) &
                (sa_runs_selected['TP'] == conf['tp'])
            ].sort_values(by='Concurrency')

        # Print table
        display(conf_df)
    
        # Plot throughputs for configuration
        plt.plot(conf_df.Thpt_per_User, conf_df.Thpt_per_GPU,
                 label=f'Replicas: {conf['rep']}  TP{conf['tp']}',
                 marker='o', markersize=4,
                 color=colors[ii%len(colors)]
                )
        for jj, val in enumerate(conf_df.Concurrency):
            plt.text(list(conf_df.Thpt_per_User)[jj],
                     list(conf_df.Thpt_per_GPU)[jj]+sa_runs_selected['Thpt_per_GPU'].max()*0.02,
                     str(val), ha='center', color=colors[ii%len(colors)])

plt.title(f'GPU: {gpu}\nModel: {model}\nISL: {isl}  OSL: {osl}')
plt.xlabel('Tok/s/User', fontsize='16')
plt.ylabel('Tok/s/GPU', fontsize='16')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.grid(True, linewidth=1, ls='--', color='gray')
plt.axis([0, None, 0, None])
plt.show()
